In [1]:
import pandas

In [3]:
import json

In [2]:
avocado = pandas.read_cvs('avocado.csv')

In [12]:
avocado.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [13]:
avocado = avocado.drop('Unnamed: 0',axis=1)

In [26]:
avocado.isna().sum()

Date            0
AveragePrice    0
Total Volume    0
4046            0
4225            0
4770            0
Total Bags      0
Small Bags      0
Large Bags      0
XLarge Bags     0
type            0
year            0
region          0
dtype: int64

In [34]:
avocado.shape

(18249, 13)

In [35]:
avocado.region.unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West', 'WestTexNewMexico'], dtype=object)

In [32]:
avocado.year.unique()

array([2015, 2016, 2017, 2018])

In [28]:
avocado_dict = avocado.to_dict(orient='records')

In [29]:
meta_field_list = []
for i in range(len(avocado)):
    # create index for each entry
    meta_field = { "index" : { "_index" : "avocado", "_type" : "_doc", "_id" : i } }
    meta_field_list.append(meta_field)

In [30]:
with open('avocado.json','w') as outfile:
    for (idx, record) in zip(meta_field_list, avocado_dict):
        json.dump(idx, outfile)
        outfile.write('\n')
        json.dump(record, outfile)
        outfile.write('\n')

```
{
  "mapping": {
    
    "properties": {
      "4046": {
        "type": "double"
      },
      "4225": {
        "type": "double"
      },
      "4770": {
        "type": "double"
      },
      "AveragePrice": {
        "type": "double"
      },
      "Date": {
        "type":   "date",
        "format": "yyyy-MM-dd"
      },
      "Large Bags": {
        "type": "double"
      },
      "Small Bags": {
        "type": "double"
      },
      "Total Bags": {
        "type": "double"
      },
      "Total Volume": {
        "type": "double"
      },
      "XLarge Bags": {
        "type": "double"
      },
      "region": {
        "type": "keyword"
      },
      "type": {
        "type": "keyword"
      },
      "year": {
        "type": "long"
      }
    }
  }
}

```

$curl -s -H "Content-Type: application/x-ndjson" -XPOST localhost:9200/avocado/_bulk --data-binary @avocado.json

## Perform queries

***1. Stats of sale volume per week in different regions in 2015***

```
GET avocado/_doc/_search?size=0
{
  "query": {
    "range": {
      "year": {
        "lt": 2016
      }
    }
  },
  "aggs": {
    "regions": {
      "terms": {
        "field": "region"
      },
      "aggs": {
        "volume_per_week": {
          "stats": {
            "field": "Total Volume"
          }
        }
      }
    }
  }
}

```

___Response:___
```
{...
    "hits" : {
    "total" : {
      "value" : 5615,
      "relation" : "eq"
    },
    "max_score" : null,
    "hits" : [ ]
  },
  "aggregations" : {
    "regions" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 4575,
      "buckets" : [
        {
          "key" : "Albany",
          "doc_count" : 104,
          "volume_per_week" : {
            "count" : 104,
            "min" : 774.2,
            "max" : 124055.31,
            "avg" : 38749.00413461539,
            "sum" : 4029896.43
          }
        },
        {
          "key" : "Atlanta",
          "doc_count" : 104,
          "volume_per_week" : {
            "count" : 104,
            "min" : 3047.38,
            "max" : 636771.37,
            "avg" : 223381.71269230772,
            "sum" : 2.323169812E7
          }
        },
        {
          "key" : "BaltimoreWashington",
          "doc_count" : 104,
          "volume_per_week" : {
            "count" : 104,
            "min" : 6884.09,
            "max" : 1007418.76,
            "avg" : 390822.8801923077,
            "sum" : 4.064557954E7
          }
        },...}
          ```
          


---------------

___2. organic avocado price ranges during holiday seasons (November & December) in 2015___
```
GET avocado/_doc/_search?size=0
{
  "query": {
    "bool": {
      "filter": [
        {
          "term": {
            "type": "organic"
          }
        },
        {
          "range": {
            "Date": {
              "gt": "2015-11-01",
              "lt":"2015-12-31"
            }
          }
        }
      ]
    }
  },
  "aggs": {
    "regions": {
      "terms": {
        "field": "region"
      },
      "aggs": {
        "organic_price_during_holiday_season": {
          "stats": {
            "field": "AveragePrice"
          }
        }
      }
    }
  }
}
```


___Response___
```
{...
"hits" : {
    "total" : {
      "value" : 431,
      "relation" : "eq"
    },
    "max_score" : null,
    "hits" : [ ]
  },
  "aggregations" : {
    "regions" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 351,
      "buckets" : [
        {
          "key" : "Albany",
          "doc_count" : 8,
          "organic_price_during_holiday_season" : {
            "count" : 8,
            "min" : 1.83,
            "max" : 1.94,
            "avg" : 1.8825,
            "sum" : 15.06
          }
        },
        {
          "key" : "Atlanta",
          "doc_count" : 8,
          "organic_price_during_holiday_season" : {
            "count" : 8,
            "min" : 1.37,
            "max" : 1.89,
            "avg" : 1.72875,
            "sum" : 13.83
          }
        },
        ...}
```

------------

___3. organic price in NewYork in different year___
```
curl -XGET "http://localhost:9200/avocado/_doc/_search?size=0" -H 'Content-Type: application/json' -d'
{
  "query": {
    "bool": {
      "filter": [
        {
          "term": {
            "type": "organic"
          }
        },
        {
          "term": {
            "region": "NewYork"
          }
        }
      ]
    }
  },
  "aggs": {
    "year_": {
      "terms": {
        "field": "year"
      },
      "aggs": {
        "organic_price": {
          "stats": {
            "field": "AveragePrice"
          }
        }
      }
    }
  }
}

```


___Response:___
```
{...
"hits" : {
    "total" : {
      "value" : 169,
      "relation" : "eq"
    },
    "max_score" : null,
    "hits" : [ ]
  },
  "aggregations" : {
    "year_" : {
      "doc_count_error_upper_bound" : 0,
      "sum_other_doc_count" : 0,
      "buckets" : [
        {
          "key" : 2017,
          "doc_count" : 53,
          "organic_price" : {
            "count" : 53,
            "min" : 1.36,
            "max" : 2.48,
            "avg" : 2.0466037735849056,
            "sum" : 108.47
          }
        },
        {
          "key" : 2015,
          "doc_count" : 52,
          "organic_price" : {
            "count" : 52,
            "min" : 1.65,
            "max" : 2.34,
            "avg" : 2.0340384615384615,
            "sum" : 105.77
          }
        },
        {
          "key" : 2016,
          "doc_count" : 52,
          "organic_price" : {
            "count" : 52,
            "min" : 1.54,
            "max" : 2.65,
            "avg" : 2.1380769230769228,
            "sum" : 111.17999999999999
          }
        },
        {
          "key" : 2018,
          "doc_count" : 12,
          "organic_price" : {
            "count" : 12,
            "min" : 1.36,
            "max" : 1.97,
            "avg" : 1.795,
            "sum" : 21.54
          }
        }
      ]
    }
  }
  ```